In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template(
    "{country_a}와 {country_b}사이의 거리는 얼마나 되니?"
)

prompt = template.format(country_a="멕시코", country_b="한국")

chat.predict(prompt)

'멕시코와 한국 사이의 직선 거리는 약 13,500km 정도입니다. 하지만 실제로 이동하는 경우에는 비행기나 배 등을 이용하여 이동해야 하므로 실제 거리는 이보다 더 길어질 수 있습니다.'

In [16]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 지리학자이다. 오직 {language}로만 대화를 한다"),
    ("ai", "안녕, {name}"), 
    ("human", "{country_a}와 {country_b}사이의 거리는 얼마나 되니? 그리고 당신의 이름은 무엇이니?")  
])

prompt = template.format_messages(
    language="korean", 
    name="조현일", 
    country_a="한국", 
    country_b="일본"
)

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 제 이름은 AI 지리학자입니다. 한국과 일본 사이의 거리는 대략 1,000km 정도입니다. 두 나라 사이에는 동해가 위치해 있어서 해상 거리로 측정됩니다. 더 궁금한 점이 있으시면 언제든지 물어보세요!')

In [20]:
from langchain.schema import BaseOutputParser 

class CommaOutputParser(BaseOutputParser): 

    def parse(self, text): 
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse("Hello, how,are,you")

['Hello', 'how', 'are', 'you']

In [27]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list생성기계입니다. 입력받은 질문들은 모두 콤마로 구분된 list로 답해야 한다 최대 {max_items}로 답해야 한다."), 
    ("human", "{question}")
])


In [28]:
chain_one = template | chat | CommaOutputParser() 


chain.invoke({
    "max_items": 5, 
    "question" : "세계적인 자동차는 무엇인가?"
})

['1. 토요타 카롤라\n2. 포드 픽업 트럭\n3. 혼다 어코드\n4. BMW 3시리즈\n5. 벤츠 E클래스']

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler 
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"), 
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, 
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"), 
    example_prompt, 
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat 

chain.invoke({
    "country": "Korea", 
})

AI: 
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

In [1]:
from langchain.chat_models import ChatOpenAI 
from langchain.prompts.few_shot import FewShotPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler 

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_template = """
    Human: {question}, 
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. The official language is French, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, picturesque countryside, and world-renowned wine regions. The country is also known for its fashion, art, and literature, with famous figures such as Coco Chanel, Claude Monet, and Victor Hugo hailing from France.

France has a long history of political and cultural influence, with a legacy that includes the French Revolution, the Enlightenment, and the development of democracy and human rights. Today, France is a member of the United Nation

'France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. The official language is French, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, picturesque countryside, and world-renowned wine regions. The country is also known for its fashion, art, and literature, with famous figures such as Coco Chanel, Claude Monet, and Victor Hugo hailing from France.\n\nFrance has a long history of political and cultural influence, with a legacy that includes the French Revolution, the Enlightenment, and the development of democracy and human rights. Today, France is a member of the United